# Task 1

In [68]:
import pandas as pd 
from math import cos, asin, sqrt

ports = pd.read_csv("ports.csv")

tracking = pd.read_csv("tracking.csv").drop_duplicates().sort_values(by='datetime', ascending=True).drop_duplicates(subset=['vessel', 'lat', 'long'], keep='first')


In [94]:
# https://stackoverflow.com/a/41337005

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    hav = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(hav))

def closest_port(data, v, ports):
    pos = min(data, key=lambda p: distance(v['lat'],v['long'],p['lat'],p['long']))
    dist = distance(pos['lat'], pos['long'], v['lat'], v['long'])
    return (ports[(ports.lat == pos['lat']) & (ports.long == pos['long'])]['port'].values[0], dist)


In [107]:
speed_threshold = 1
ports_dict = ports[['lat', 'long']].to_dict('records')
ports_km_threshold = 3

started_port = 0
voyages_list = []

for vessel in tracking.drop_duplicates(subset=['vessel'])['vessel'].values:
    for index, row in tracking[tracking.vessel == vessel].iterrows():
        if started_port == 0 and row['speed'] < speed_threshold:
            #record startport
            started_port = 1
            (begin_port_id, _) = closest_port(ports_dict, {'lat': row['lat'], 'long': row['long']}, ports)
        elif started_port == 1 and row['speed'] > speed_threshold:
            #record start date 
            started_port = 2
            begin_date = row['datetime']
        elif started_port == 2 and row['speed'] < speed_threshold:
            #record end date, end port, add to new df
            end_date = row['datetime']
            (end_port_id, closest_dist) = closest_port(ports_dict, {'lat': row['lat'], 'long': row['long']}, ports)
            if ports_km_threshold > closest_dist:
                voyages_list.append({'vessel': vessel, 'begin_date': begin_date, 'end_date': end_date, 'begin_port_id': begin_port_id, 'end_port_id': end_port_id})
                started_port = 0
voyages = pd.DataFrame(voyages_list)
voyages = voyages[voyages['begin_port_id'] != voyages['end_port_id']]

In [123]:
voyages.to_csv('voyages.csv', index=False)
